In [1]:
import json
import re
from collections import Counter
from collections import defaultdict

In [2]:
with open('/home/thoang/workspace/spert/data/datasets/ade/ade_full.json', 'r') as f:
    ade = json.load(f)

In [3]:
len(ade)

4272

## Confirmation 1: ade accepts relation_flag

In [4]:
# ade accepts relation_flag
count = 0
for i, row in enumerate(ade):
    if len(row['relations']) == 0:
        count += 1
count

0

## Confirmation 2: Set of relation are Cartesian product of drugs and adverse-effects

In [6]:
count = 0
for i, row in enumerate(ade):
    tokens = row['tokens']
    entities = row['entities']
    relations = row['relations']
    drugs = [e for e in entities if e['type'] == 'Drug']
    adversed = [e for e in entities if e['type'] == 'Adverse-Effect']
    if len(relations) != (len(drugs) * len(adversed)):
        print(i)
        print(tokens)
        for e in row['entities']:
            start, end, etype = e['start'], e['end'], e['type']
            w = ' '.join(tokens[start: end])
            print(f"{etype}({w})-({start},{end})")
        print('+++++')
        
        for r in row['relations']:
            head, tail, rtype = entities[r['head']], entities[r['tail']], r['type']
            hw = ' '.join(tokens[head['start']: head['end']])
            tw = ' '.join(tokens[tail['start']: tail['end']])
            print(f"{rtype}({hw}, {tw}) - ({head['start']},{head['end']},{tail['start']},{tail['end']})")
        count += 1
        print('------------------')

81
['The', 'purpose', 'of', 'this', 'report', 'is', 'to', 'document', 'a', 'new', 'case', 'of', 'in', 'utero', 'CP', 'exposure', 'with', 'multiple', 'congenital', 'anomalies', 'and', 'to', 'establish', 'an', 'apparent', 'CP', 'embryopathy', 'phenotype', '.']
Adverse-Effect(embryopathy)-(26,27)
Drug(CP)-(25,26)
Adverse-Effect(multiple congenital anomalies)-(17,20)
Drug(CP)-(14,15)
+++++
Adverse-Effect(embryopathy, CP) - (26,27,25,26)
Adverse-Effect(multiple congenital anomalies, CP) - (17,20,14,15)
------------------
164
['The', 'present', 'report', 'illustrates', 'a', 'rare', 'case', 'of', 'refractory', 'akathisia', 'after', 'interferon', '-', 'alpha', 'treatment', 'and', 'also', 'that', 'levodopa', 'treatment', 'would', 'be', 'theoretically', 'and', 'practically', 'useful', 'in', 'reducing', 'the', 'neurotoxicity', 'associated', 'with', 'interferon', '-', 'alpha', '.']
Adverse-Effect(akathisia)-(9,10)
Drug(interferon - alpha)-(11,14)
Adverse-Effect(neurotoxicity)-(29,30)
Drug(interfer

In [7]:
count

83

In [9]:
count/len(ade) * 100

1.942883895131086

## Confirmation 3: 

In [10]:
count = 0
for i, row in enumerate(ade):
    tokens = row['tokens']
    entities = row['entities']
    relations = row['relations']
    drugs = [e for e in entities if e['type'] == 'Drug']
    adversed = [e for e in entities if e['type'] == 'Adverse-Effect']
    print(i)
    print(tokens)
    for e in row['entities']:
        start, end, etype = e['start'], e['end'], e['type']
        w = ' '.join(tokens[start: end])
        print(f"{etype}({w})-({start},{end})")
    print('+++++')

    for r in row['relations']:
        head, tail, rtype = entities[r['head']], entities[r['tail']], r['type']
        hw = ' '.join(tokens[head['start']: head['end']])
        tw = ' '.join(tokens[tail['start']: tail['end']])
        print(f"{rtype}({hw}, {tw}) - ({head['start']},{head['end']},{tail['start']},{tail['end']})")
    count += 1
    print('------------------')

0
['Intravenous', 'azithromycin', '-', 'induced', 'ototoxicity', '.']
Adverse-Effect(ototoxicity)-(4,5)
Drug(azithromycin)-(1,2)
+++++
Adverse-Effect(ototoxicity, azithromycin) - (4,5,1,2)
------------------
1
['Immobilization', ',', 'while', 'Paget', "'s", 'bone', 'disease', 'was', 'present', ',', 'and', 'perhaps', 'enhanced', 'activation', 'of', 'dihydrotachysterol', 'by', 'rifampicin', ',', 'could', 'have', 'led', 'to', 'increased', 'calcium', '-', 'release', 'into', 'the', 'circulation', '.']
Adverse-Effect(increased calcium - release)-(23,27)
Drug(dihydrotachysterol)-(15,16)
+++++
Adverse-Effect(increased calcium - release, dihydrotachysterol) - (23,27,15,16)
------------------
2
['Unaccountable', 'severe', 'hypercalcemia', 'in', 'a', 'patient', 'treated', 'for', 'hypoparathyroidism', 'with', 'dihydrotachysterol', '.']
Adverse-Effect(hypercalcemia)-(2,3)
Drug(dihydrotachysterol)-(10,11)
+++++
Adverse-Effect(hypercalcemia, dihydrotachysterol) - (2,3,10,11)
------------------
3
['ME

------------------
565
['There', 'have', 'been', 'more', 'than', '20', 'observations', 'of', 'the', 'appearance', 'or', 'aggravation', 'of', 'this', 'granulomatosis', 'with', 'interferon', 'alfa', 'and', 'more', 'recently', 'with', 'the', 'combination', 'of', 'interferon', 'alfa', 'plus', 'ribavirin', '.']
Adverse-Effect(granulomatosis)-(14,15)
Drug(interferon alfa)-(16,18)
Drug(interferon alfa)-(25,27)
Drug(ribavirin)-(28,29)
+++++
Adverse-Effect(granulomatosis, interferon alfa) - (14,15,16,18)
Adverse-Effect(granulomatosis, interferon alfa) - (14,15,25,27)
Adverse-Effect(granulomatosis, ribavirin) - (14,15,28,29)
------------------
566
['We', 'report', 'two', 'new', 'cases', 'of', 'sarcoidosis', 'in', 'two', 'patients', 'with', 'hepatitis', 'C', 'virus', 'infection', 'treated', 'with', 'interferon', 'alfa', 'and', 'ribavirin', '.']
Adverse-Effect(sarcoidosis)-(6,7)
Drug(interferon alfa)-(17,19)
Drug(ribavirin)-(20,21)
+++++
Adverse-Effect(sarcoidosis, interferon alfa) - (6,7,17,19)
A

Adverse-Effect(Peyronie 's disease)-(2,5)
Drug(colchicine)-(9,10)
+++++
Adverse-Effect(Peyronie 's disease, colchicine) - (2,5,9,10)
------------------
1051
['However', 'here', 'we', 'reported', 'two', 'patients', ',', 'presenting', 'with', 'PD', 'during', 'high', 'dose', 'colchicine', 'treatment', 'for', 'familiar', 'mediterranean', 'fever', '(', 'FMF', ')', '.']
Adverse-Effect(PD)-(9,10)
Drug(colchicine)-(13,14)
+++++
Adverse-Effect(PD, colchicine) - (9,10,13,14)
------------------
1052
['A', 'patient', 'developed', 'cholestatic', 'hepatitis', 'while', 'being', 'treated', 'with', 'nitrofurantoin', '.']
Adverse-Effect(cholestatic hepatitis)-(3,5)
Drug(nitrofurantoin)-(9,10)
+++++
Adverse-Effect(cholestatic hepatitis, nitrofurantoin) - (3,5,9,10)
------------------
1053
['A', 'second', 'episode', 'of', 'jaundice', 'followed', 'the', 'intravaginal', 'administration', 'of', 'a', 'mixture', 'of', 'furazolidone', 'and', 'nifuroxime', '.']
Adverse-Effect(jaundice)-(4,5)
Drug(furazolidone)-(

------------------
1583
['Several', 'hypersensitivity', 'reactions', 'to', 'cloxacillin', 'have', 'been', 'reported', ',', 'although', 'IgE', '-', 'mediated', 'allergic', 'reactions', 'to', 'the', 'drug', 'are', 'rare', 'and', 'there', 'is', 'little', 'information', 'about', 'possible', 'tolerance', 'to', 'other', 'semisynthetic', 'penicillins', 'or', 'cephalosporins', 'in', 'patients', 'with', 'cloxacillin', 'allergy', '.']
Adverse-Effect(hypersensitivity)-(1,2)
Drug(cloxacillin)-(4,5)
Adverse-Effect(IgE - mediated allergic reactions)-(10,15)
+++++
Adverse-Effect(hypersensitivity, cloxacillin) - (1,2,4,5)
Adverse-Effect(IgE - mediated allergic reactions, cloxacillin) - (10,15,4,5)
------------------
1584
['Cardiac', 'toxicity', 'related', 'to', 'BCNU', 'has', 'not', 'been', 'described', 'well', '.']
Adverse-Effect(Cardiac toxicity)-(0,2)
Drug(BCNU)-(4,5)
+++++
Adverse-Effect(Cardiac toxicity, BCNU) - (0,2,4,5)
------------------
1585
['Myocardial', 'ischemia', 'associated', 'with', 'h

+++++
Adverse-Effect(coronary artery disease, citalopram) - (30,33,14,15)
Adverse-Effect(coronary artery disease, propafenone) - (30,33,12,13)
Adverse-Effect(dizziness, citalopram) - (24,25,14,15)
Adverse-Effect(dizziness, propafenone) - (24,25,12,13)
Adverse-Effect(falls, citalopram) - (26,27,14,15)
Adverse-Effect(falls, propafenone) - (26,27,12,13)
------------------
2120
['Three', 'months', 'after', 'starting', 'citalopram', ',', 'she', 'experienced', 'episodes', 'of', 'chest', 'tightness', 'and', 'dizziness', '.']
Adverse-Effect(chest tightness)-(10,12)
Drug(citalopram)-(4,5)
Adverse-Effect(dizziness)-(13,14)
+++++
Adverse-Effect(chest tightness, citalopram) - (10,12,4,5)
Adverse-Effect(dizziness, citalopram) - (13,14,4,5)
------------------
2121
['CONCLUSIONS', ':', 'Clinicians', 'should', 'be', 'aware', 'of', 'this', 'adverse', 'reaction', 'when', 'facing', 'similar', 'complex', 'neurologic', 'symptoms', 'in', 'patients', 'who', 'are', 'receiving', 'the', 'antibiotic', 'treatment

Adverse-Effect(portal hypertension, busulphan) - (21,23,8,9)
------------------
2655
['Although', 'a', 'lot', 'of', 'amiodarone', '-', 'induced', 'torsades', 'de', 'pointe', 'have', 'been', 'published', ',', 'a', 'review', 'of', 'all', 'these', 'cases', 'and', 'the', 'underlying', 'risk', '-', 'factors', 'has', 'never', 'been', 'made', '.']
Adverse-Effect(torsades de pointe)-(7,10)
Drug(amiodarone)-(4,5)
+++++
Adverse-Effect(torsades de pointe, amiodarone) - (7,10,4,5)
------------------
2656
['Amiodarone', 'induced', 'torsades', 'de', 'pointe', '.']
Adverse-Effect(torsades de pointe)-(2,5)
Drug(Amiodarone)-(0,1)
+++++
Adverse-Effect(torsades de pointe, Amiodarone) - (2,5,0,1)
------------------
2657
['We', 'report', 'three', 'cases', 'of', 'amiodarone', '-', 'induced', 'torsades', 'de', 'pointe', '.']
Adverse-Effect(torsades de pointe)-(8,11)
Drug(amiodarone)-(5,6)
+++++
Adverse-Effect(torsades de pointe, amiodarone) - (8,11,5,6)
------------------
2658
['The', 'syndrome', 'of', 'irre

------------------
3201
['A', '61', 'year', '-', 'old', 'male', 'patient', 'developed', 'gynecomastia', 'after', 'starting', 'theophylline', '.']
Adverse-Effect(gynecomastia)-(8,9)
Drug(theophylline)-(11,12)
+++++
Adverse-Effect(gynecomastia, theophylline) - (8,9,11,12)
------------------
3202
['Gynecomastia', 'associated', 'with', 'theophylline', '.']
Adverse-Effect(Gynecomastia)-(0,1)
Drug(theophylline)-(3,4)
+++++
Adverse-Effect(Gynecomastia, theophylline) - (0,1,3,4)
------------------
3203
['No', 'other', 'endogenous', 'or', 'exogenous', 'cause', 'for', 'the', 'gynecomastia', 'could', 'be', 'found', 'except', 'for', 'the', 'theophylline', '.']
Adverse-Effect(gynecomastia)-(8,9)
Drug(theophylline)-(15,16)
+++++
Adverse-Effect(gynecomastia, theophylline) - (8,9,15,16)
------------------
3204
['The', 'gynecomastia', 'regressed', 'when', 'the', 'theophylline', 'was', 'discontinued', '.']
Adverse-Effect(gynecomastia)-(1,2)
Drug(theophylline)-(5,6)
+++++
Adverse-Effect(gynecomastia, the

Drug(pimozide)-(7,8)
+++++
Adverse-Effect(Acute dystonic reaction, pimozide) - (0,3,7,8)
------------------
3707
['This', 'paper', 'reports', 'on', 'a', '6.9-year', '-', 'old', 'autistic', 'male', 'who', 'developed', 'repeated', 'episodes', 'of', 'acute', 'dystonic', 'reactions', 'associated', 'with', 'pimozide', 'administration', 'at', 'the', 'doses', 'of', '0.096', 'mg', '/', 'kg', '/', 'day', 'and', '0.032', 'mg', '/', 'kg', '/', 'day', 'and', '32', 'hours', 'following', 'pimozide', 'withdrawal', ',', 'as', 'well', 'as', 'during', 'subsequent', 'thioridazine', 'administration', '.']
Adverse-Effect(acute dystonic reactions)-(15,18)
Drug(pimozide)-(20,21)
Drug(pimozide)-(43,44)
Drug(thioridazine)-(51,52)
+++++
Adverse-Effect(acute dystonic reactions, pimozide) - (15,18,20,21)
Adverse-Effect(acute dystonic reactions, pimozide) - (15,18,43,44)
Adverse-Effect(acute dystonic reactions, thioridazine) - (15,18,51,52)
------------------
3708
['Although', 'there', 'is', 'one', 'case', 'report

Adverse-Effect(prolongation of the QT interval)-(19,24)
Drug(pirmenol)-(39,40)
Adverse-Effect(T wave inversion)-(25,28)
+++++
Adverse-Effect(prolongation of the QT interval, pirmenol) - (19,24,39,40)
Adverse-Effect(T wave inversion, pirmenol) - (25,28,39,40)
------------------
4242
['Disseminated', 'muscular', 'cysticercosis', 'with', 'myositis', 'induced', 'by', 'praziquantel', 'therapy', '.']
Adverse-Effect(myositis)-(4,5)
Drug(praziquantel)-(7,8)
+++++
Adverse-Effect(myositis, praziquantel) - (4,5,7,8)
------------------
4243
['We', 'describe', 'a', 'case', 'of', 'disseminated', 'muscular', 'cysticercosis', 'followed', 'by', 'myositis', '(', 'fever', ',', 'diffuse', 'myalgia', ',', 'weakness', 'of', 'the', 'lower', 'limbs', ',', 'and', 'inflammatory', 'reaction', 'around', 'dying', 'cysticerci', ')', 'induced', 'by', 'praziquantel', 'therapy', ',', 'an', 'event', 'not', 'described', 'previously', '.']
Adverse-Effect(diffuse myalgia)-(14,16)
Drug(praziquantel)-(32,33)
Adverse-Effect(